In [58]:
### Loading Credentials from local file; this cell is meant to be deleted before publishing
import yaml

with open("../creds.yml", 'r') as ymlfile:
    cfg = yaml.safe_load(ymlfile)

uri = cfg["sonar_creds"]["uri"]
user = cfg["sonar_creds"]["user"]
password = cfg["sonar_creds"]["pass"]

# Project Summary 


## Citing SoNAR (IDH)

# Data Description


The SoNAR (IDH) database consists of nodes and edges. Each of the nodes and edges have additional attributes that provide rich meta information. 

This data description section provides details about the data sources and overall characteristics of the data. The section is based on the state of the SoNAR (IDH) database during February 2021. 

More in depth details about the data, the database and the data preparation will get published soon. 

## Summary Stats

The SoNAR (IDH) database has the following aggregated characteristics:

**Nodes Summary**

* 9 categories of Nodes
* 34.511.952 Nodes

|Node Type | Node Count |
|---------------|------------|
| CorpName      | 1.487.711  |
| GeoName       | 308.197    |
| MeetName      | 814.044    |
| PerName       | 5.087.660  |
| TopicTerm     | 212.135    |
| UniTitle      | 385.300    |
| ChronTerm     | 537.054    |
| IsilTerm      | 611        |
| Resource      | 25.679.240 |

**Edges Summary**

* 98.530.160 Edges
* 9 categories of Edges


| Edge Type       | Edge Count     |
|---------------------|------------|
| RelationToPerName   | 14.630.465 |
| RelationToCorpName  | 5.099.190  |
| RelationToMeetName  | 263.180    |
| RelationToUniTitle  | 53.998     |
| RelationToTopicTerm | 4.951.617  |
| RelationToGeoName   | 5.140.556  |
| RelationToChronTerm | 5.446.841  |
| RelationToIsil      | 55.556.913 |
| RelationToResource  | 7.387.400  |
| socialRelation      | 40.301.595 |

## Data Sources 

SoNAR (IDH) combines data from four different data sources. The table below provides a compact overview:


|Data Source |  Number of Nodes | Number of Edges <br>(incl. *RelationToIsilTerm*) |
|--------|----------| -------- |
|[GND (*Integrated Authority File*)](https://www.dnb.de/EN/Professionell/Standardisierung/GND/gnd_node.html)| 8.295.047 | 32.776.628 |
|[DNB (*German National Library*)](https://www.dnb.de/EN/Home/home_node.html)|19.384.733| 5.655.859 |
|[ZDB (*Zeitschriftendatenbank*)](https://www.zeitschriftendatenbank.de/startseite/)|1.908.334| 43.419.339 |
|KPE|4.386.173| 16.678.334 |

# Data Access

We will need some specific libraries to work with the SoNAR (IDH) database. Let's start with installing the `neo4j` library.

When you are using the curriculum on binder or locally or by running it as a docker container locally, the pacckage is already installed. When you want to interact with the SoNAR (IDH) database independently install the package with the following code line in a new notebook cell:

```python
!pip install neo4j
```

In [350]:
from neo4j import GraphDatabase

driver = GraphDatabase.driver(uri, auth=(user, password))

With the call above we create a [Neo4j driver object](https://neo4j.com/docs/api/python-driver/current/api.html#driver). This driver now stores the connection details for the database. We can use this driver now to send requests to the database for example to request data.

# Data Exploration

Data exploration is usually the very first thing to do when working with new data. So let's start diving into the SoNAR (IDH) database by exploring it. 

Whenever we want to retrieve data from the Neo4j database of SoNAR (IDH) we can use a query language called "Cypher Query Language". Cypher provides a rather easy to comprehend syntax for requesting data from the database.

Throughout this curriculum we will use this Cypher Query Language whenever we directly retrieve data from SoNAR (IDH).

## Nodes

### Node Labels

Let's start off with a simple query. Let's request the database to return all [node labels](https://neo4j.com/docs/getting-started/current/graphdb-concepts/#graphdb-labels). Node labels are categories the nodes can belong to. You can think of them as entity groups. The SoNAR (IDH) database distinguishes between Persons, Corporations and more. Let's ask the database it self to return all the labels available. 

In [60]:
with driver.session() as session:
    result = session.run("CALL db.labels()").data()
    
result

[{'label': 'CorpName'},
 {'label': 'GeoName'},
 {'label': 'MeetName'},
 {'label': 'PerName'},
 {'label': 'TopicTerm'},
 {'label': 'UniTitle'},
 {'label': 'ChronTerm'},
 {'label': 'IsilTerm'},
 {'label': 'Resource'}]

**Code Breakdown:**

>The `with` statement is basically used to make the database call as resource effective and concise as possible. There are more advantages of the `with` call but their explanation would exceed the goal of this curriculum. However, an in-depth explanation of the `with` statement can be found [here](https://www.python.org/dev/peps/pep-0343/).
>
>When we request data from the database we need to establish a connection (`session`). The `driver` object we created earlier stores the connection details. When we use the method `driver.session()` we establish a new connection. This connection is assigned to the object `session` object for the `while` statement.
>
>The most relevant part of the code for retrieving the data is `"CALL db.labels()"`. This part is the actual Cypher query. The `CALL` clause is used to call the `db.labels()` procedure. More details about Neo4j procedures can be found below.
>
>The result of this code chunk is a list that contains a key-value pair (`dictionary`) per label in the database.  

Some useful built-in procedures for exploring and describing the database are listed in the table below. You can get a full list of built-in procedures by using the following query: `CALL dbms.procedures()`


|Procedure | Description |
|---------|----------|
|`db.labels()`| List all labels in the database.|
|`db.propertyKeys()`|List all property keys in the database.|
|`db.relationshipTypes`|List all relationship types in the database. |
|`db.schema`| Show the schema of the data. |
|`db.stats.retrieve`|Retrieve statistical data about the current database. <br>Valid sections are 'GRAPH COUNTS', 'TOKENS', 'QUERIES', 'META'|

### Selecting Nodes

You can select nodes by using the `MATCH` statement. Cypher uses `ASCII-art` syntax to define nodes, relationships and the direction of relationships in queries. 

Nodes are referred to by using parentheses `()`. Inside the parentheses you can define a Node variable. This variable can be used to refer to a specific set of nodes throughout the rest of the query.

The example below matches any kind of Node and assigns the variable name n `(n)`. We use the `LIMIT` statement to tell the database we only want to have the first 5 results.

In [138]:
query = """
MATCH (n)
RETURN n
LIMIT 5
"""

In [139]:
import pandas as pd

with driver.session() as session:
    result = session.run(query).data()
    
result

[{'n': {'GenType': 'b',
   'DateOriginal': '1951-',
   'SpecType': 'kiz',
   'VariantName': 'CNS;;;CNS',
   'DateApproxBegin': '1951',
   'Id': '(DE-588)110-7',
   'id': 'Aut110_7',
   'Uri': 'http://d-nb.info/gnd/110-7',
   'Name': 'Congress of Neurological Surgeons'}},
 {'n': {'GenType': 'b',
   'Id': '(DE-588)191-0',
   'id': 'Aut191_0',
   'Uri': 'http://d-nb.info/gnd/191-0',
   'SubUnit': 'División de Ciencias Matemáticas, Médicas y de la Naturaleza',
   'Name': 'Consejo Superior de Investigaciones Científicas'}},
 {'n': {'GenType': 'b',
   'Id': '(DE-588)257-4',
   'id': 'Aut257_4',
   'SpecType': 'kiz',
   'Uri': 'http://d-nb.info/gnd/257-4',
   'Name': 'Fondazione Antonio Baselli'}},
 {'n': {'GenType': 'b',
   'Id': '(DE-588)273-2',
   'id': 'Aut273_2',
   'SpecType': 'wit',
   'Uri': 'http://d-nb.info/gnd/273-2',
   'Name': 'Copyright Law Symposium'}},
 {'n': {'GenType': 'b',
   'DateOriginal': '1865-',
   'VariantName': "Kornel'skii Universitet;;;Cornell Univ.;;;Universit

### Filtering Nodes

In [140]:
query = """
MATCH (n:PerName)
RETURN n
LIMIT 1"""

In [142]:
with driver.session() as session:
    result = session.run(query).data()
    
result

[{'n': {'GenType': 'p',
   'VariantName': 'Lombez, Ambrosius de;;;La Peirie, Ambroise;;;LaPeirie, Ambroise;;;Ambroise;;;Lombez, Ambroise de;;;LaPeyrie;;;Lombez, Ambrosius von',
   'SpecType': 'piz',
   'Id': '(DE-588)100000096',
   'id': 'Aut100000096',
   'Uri': 'http://d-nb.info/gnd/100000096',
   'Name': 'Ambrosius'}}]

**Filtering Nodes by Properties**

Now, let's try to find the node of Max Weber, the sociologist and political economist. 
We can define a filter based on properties of a node. The query below only returns nodes that have "Weber, Max" as `Name` property.

Also, we return the count of nodes `RETURN count(n)` not the actual nodes. We suspect the name "Weber, Max" to be not unique inside the big SoNAR (IDH) database. So we want to check, how many Max Webers we can find.  

In [143]:
query = """
MATCH (n:PerName {Name: 'Weber, Max'})
RETURN count(n)
"""

In [144]:
with driver.session() as session:
    result = session.run(query).data()
    
result

[{'count(n)': 34}]

In fact we detected 34 hits in the database. So we need to apply more filters to find the correct Max.

Let's start by checking what properties are available for nodes of type `PerName`:

In [145]:
query = """
MATCH (n:PerName)
WITH LABELS(n) AS labels , KEYS(n) AS keys
UNWIND labels AS label
UNWIND keys AS key
RETURN DISTINCT label, COLLECT(DISTINCT key) AS props
ORDER BY label
"""

with driver.session() as session:
    result = session.run(query).data()
    
result

[{'label': 'PerName',
  'props': ['Uri',
   'Name',
   'Id',
   'VariantName',
   'id',
   'GenType',
   'SpecType',
   'DateStrictOriginal',
   'DateStrictEnd',
   'DateApproxOriginal',
   'DateApproxEnd',
   'DateStrictBegin',
   'Gender',
   'DateApproxBegin',
   'OldId']}]

We can see that there are several `date` properties for `PerName` nodes. The year of birth is stored in the property called `DateApproxBegin`. 
So let's apply a date filter. Let's assume we only know that Max Weber was born in the year 1864 and we want to filter based on this information.

In [146]:
query = """
MATCH (n:PerName)
WHERE n.Name = "Weber, Max" AND n.DateApproxBegin = "1864"
RETURN n
"""

with driver.session() as session:
    result = session.run(query).data()
    
result

[{'n': {'GenType': 'p',
   'DateApproxEnd': '1920',
   'DateStrictOriginal': '21.04.1864-14.06.1920',
   'VariantName': 'Makesi, Weipei;;;Weber, Karl Emil Maximilian;;;Veber, Maks;;;Veber, M.;;;Weibo, ...;;;Uēbā, Makkusu;;;Wibir, Māks;;;Weibo, Makesi;;;Fībir, Māks;;;Vēbā, Makkusu;;;Ma ke si Wei bo;;;Makesi-Weibo;;;馬克思, 威培;;;فيبر، ماكس;;;マックス・ウェーバー;;;马克斯•韦伯;;;ובר, מקס;;;韦伯, 马克斯',
   'SpecType': 'piz',
   'DateStrictBegin': '21.04.1864',
   'DateStrictEnd': '14.06.1920',
   'Gender': '1',
   'DateApproxOriginal': '1864-1920',
   'Uri': 'http://d-nb.info/gnd/118629743',
   'Name': 'Weber, Max',
   'DateApproxBegin': '1864',
   'Id': '(DE-588)118629743',
   'id': 'Aut118629743'}}]

As a last example, let's assume we only know that the last Name of Max Weber is spelled "韦伯" in Chinese, so we need to use this information as a filter. 

In the query result above, you see a node property called `VariantName`. This variable stores many alternative variants of the name we are looking for. So let's check how we could query the database by searching within this property:

In [149]:
query = """
MATCH (n:PerName)
WHERE n.VariantName CONTAINS "韦伯"
RETURN n
"""

with driver.session() as session:
    result = session.run(query).data()
    
result

[{'n': {'GenType': 'p',
   'DateApproxEnd': '1920',
   'DateStrictOriginal': '21.04.1864-14.06.1920',
   'VariantName': 'Makesi, Weipei;;;Weber, Karl Emil Maximilian;;;Veber, Maks;;;Veber, M.;;;Weibo, ...;;;Uēbā, Makkusu;;;Wibir, Māks;;;Weibo, Makesi;;;Fībir, Māks;;;Vēbā, Makkusu;;;Ma ke si Wei bo;;;Makesi-Weibo;;;馬克思, 威培;;;فيبر، ماكس;;;マックス・ウェーバー;;;马克斯•韦伯;;;ובר, מקס;;;韦伯, 马克斯',
   'SpecType': 'piz',
   'DateStrictBegin': '21.04.1864',
   'DateStrictEnd': '14.06.1920',
   'Gender': '1',
   'DateApproxOriginal': '1864-1920',
   'Uri': 'http://d-nb.info/gnd/118629743',
   'Name': 'Weber, Max',
   'DateApproxBegin': '1864',
   'Id': '(DE-588)118629743',
   'id': 'Aut118629743'}},
 {'n': {'GenType': 'p',
   'VariantName': 'Veber, Mattias;;;Weber, Matthias;;;Weibo, Madiyasi;;;Ma di ya si Wei bo;;;Madiyasi-Weibo;;;Ma ti ya si Wei bo;;;Matiyasi-Weibo;;;Weibo, Matiyasi;;;马蒂亚斯•韦伯;;;韦伯, 马蒂亚斯',
   'SpecType': 'piz',
   'DateApproxBegin': '1967',
   'Gender': '1',
   'DateApproxOriginal': 

## Relationship

### Relationships Types

Similar to node labels we can retrieve the categories of the relations inside the database. Every relation must have exactly one relationship type. This type defines the kind or category the relation belongs to. 

In [18]:
with driver.session() as session:
    result = session.run("CALL db.relationshipTypes()").data()

    
pd.DataFrame(result)

,relationshipType
0,RelationToIsilTerm
1,RelationToChronTerm
2,RelationToCorpName
3,RelationToTopicTerm
4,RelationToGeoName
5,SocialRelation
6,RelationToMeetName
7,RelationToPerName
8,RelationToUniTitle
9,RelationToResource


### Selecting Relationships

In the section about nodes we saw that we need to use parenthesis (`()`) to select nodes. When selecting relationships on the other hand we need to use brackets (`[]`) instead.

Additionally we can not solely query for plain relationships, but we need to define a pattern in which this relationship needs to appear in the database. 

The most simple relationship pattern we can define is, the relationship needs to be between any kind two nodes. In the cypher language this would be expressed as:
>`()-[r]-()`


In [159]:
query = """
MATCH ()-[r]-()
RETURN r
LIMIT 5
"""

with driver.session() as session:
    result = session.run(query).data()

    
result

[{'r': ({}, 'RelationToCorpName', {})},
 {'r': ({}, 'RelationToCorpName', {})},
 {'r': ({}, 'RelationToCorpName', {})},
 {'r': ({}, 'RelationToCorpName', {})},
 {'r': ({}, 'RelationToCorpName', {})}]

### Filtering Relationships

You can filter relationships in a similar fashion like you can filter nodes. 
Let's retrieve relationships of the type `SocialRelation`.

In [346]:
from neo4j import GraphDatabase

driver = GraphDatabase.driver(uri, auth=(user, password))

In [299]:
query = """
MATCH ()-[r:SocialRelation]-()
RETURN r
LIMIT 5
"""

with driver.session() as session:
    result = session.run(query).data()

    
result

[{'r': ({}, 'SocialRelation', {})},
 {'r': ({}, 'SocialRelation', {})},
 {'r': ({}, 'SocialRelation', {})},
 {'r': ({}, 'SocialRelation', {})},
 {'r': ({}, 'SocialRelation', {})}]

**Filtering Relationships by Properties**

In [334]:
query = """
MATCH p = ()-[r:SocialRelation]-()
UNWIND relationships(p) as rel
RETURN properties(rel) as properties
LIMIT 5
"""

with driver.session() as session:
    result = session.run(query).data()

    
result

[{'properties': {'TypeAddInfo': 'undirected',
   'SourceType': 'associatedRelation',
   'Source': 'Bib(DE_588)5550944_7'}},
 {'properties': {'TypeAddInfo': 'undirected',
   'SourceType': 'associatedRelation',
   'Source': 'Bib(DE_588)5550944_7'}},
 {'properties': {'TypeAddInfo': 'undirected',
   'SourceType': 'associatedRelation',
   'Source': 'Bib(DE_588)16249939_5'}},
 {'properties': {'TypeAddInfo': 'undirected',
   'SourceType': 'associatedRelation',
   'Source': 'Bib(DE_588)16249939_5'}},
 {'properties': {'TypeAddInfo': 'undirected',
   'SourceType': 'associatedRelation',
   'Source': 'BibBVBBV044242543'}}]

In fact, the properties of relationships of the type `SocialRelation` have three different elements: 

* `TypeAddInfo` - either **directed** or **undirected**
* `SourceType` - can take the values: **associatedRelation**, **areCoAuthors**, **areCoEditors**, **affiliatedRelation**, **correspondedRelation**, **knows**
* `Source` - **id** of the source

Let's retrieve some people that are connected to each other because they had a correspondence with each other.

In [351]:
query = """
MATCH (n1:PerName)-[r:SocialRelation]-(n2:PerName)
WHERE r.SourceType = "correspondedRelation"
RETURN n1.Name, n2.Name, r.SourceType, r.TypeAddInfo
LIMIT 5
"""

with driver.session() as session:
    result = session.run(query).data()

    
result

[{'n1.Name': 'André, Christian Carl',
  'n2.Name': 'Richter, Karl Friedrich Enoch',
  'r.SourceType': 'correspondedRelation',
  'r.TypeAddInfo': 'directed'},
 {'n1.Name': 'André, Christian Carl',
  'n2.Name': 'Hügel, Elisabeth von',
  'r.SourceType': 'correspondedRelation',
  'r.TypeAddInfo': 'directed'},
 {'n1.Name': 'André, Christian Carl',
  'n2.Name': 'Sturm, Karl Christian Gottlob',
  'r.SourceType': 'correspondedRelation',
  'r.TypeAddInfo': 'directed'},
 {'n1.Name': 'Ring, Friedrich Dominicus',
  'n2.Name': 'Pfeffel, Gottlieb Konrad',
  'r.SourceType': 'correspondedRelation',
  'r.TypeAddInfo': 'directed'},
 {'n1.Name': 'Bothe, Friedrich Heinrich',
  'n2.Name': 'Hauff, Hermann',
  'r.SourceType': 'correspondedRelation',
  'r.TypeAddInfo': 'directed'}]

We can see that all of these relationships have a `TypeAddInfo` of **directed**. Relationships can be directed and undirected. In the SoNAR (IDH) database all correspondences are directed and therefor hold the information whether someone was contacted or contacted someone else. 

Let's see who received letters from Max Weber. The query below extends the basic `()-[]-()` structure for representing a node-relationship search pattern by an `>`. This arrow defines that we are searching only for directed relationships. So the new pattern scaffold is `()-[]->()`

In [353]:
query = """
MATCH (n1:PerName)-[r:SocialRelation]->(n2:PerName)
WHERE n1.Name = "Weber, Max" AND n1.DateApproxBegin = "1864" 
AND r.SourceType = "correspondedRelation"
RETURN n1.Name, n2.Name, r.SourceType, r.TypeAddInfo
"""

with driver.session() as session:
    result = session.run(query).data()
    
result

[{'n1.Name': 'Weber, Max',
  'n2.Name': 'Lukács, Georg',
  'r.SourceType': 'correspondedRelation',
  'r.TypeAddInfo': 'directed'},
 {'n1.Name': 'Weber, Max',
  'n2.Name': 'Susman, Margarete',
  'r.SourceType': 'correspondedRelation',
  'r.TypeAddInfo': 'directed'},
 {'n1.Name': 'Weber, Max',
  'n2.Name': 'Wolfskehl, Karl',
  'r.SourceType': 'correspondedRelation',
  'r.TypeAddInfo': 'directed'},
 {'n1.Name': 'Weber, Max',
  'n2.Name': 'Jaspers, Karl',
  'r.SourceType': 'correspondedRelation',
  'r.TypeAddInfo': 'directed'},
 {'n1.Name': 'Weber, Max',
  'n2.Name': 'Jaffe, Else',
  'r.SourceType': 'correspondedRelation',
  'r.TypeAddInfo': 'directed'},
 {'n1.Name': 'Weber, Max',
  'n2.Name': 'Ernst, Paul',
  'r.SourceType': 'correspondedRelation',
  'r.TypeAddInfo': 'directed'},
 {'n1.Name': 'Weber, Max',
  'n2.Name': 'Diederichs, Eugen',
  'r.SourceType': 'correspondedRelation',
  'r.TypeAddInfo': 'directed'}]

## Summary Cypher Query Language

# Descriptive Analyses

## General Database Summary

In [20]:
query = """
match (n) return 'Number of Nodes: ' + count(n) as output UNION
match ()-[]->() return 'Number of Relationships: ' + count(*) as output UNION
CALL db.labels() YIELD label RETURN 'Number of Labels: ' + count(*) AS output UNION
CALL db.relationshipTypes() YIELD relationshipType  RETURN 'Number of Relationship Types: ' + count(*) AS output
"""

with driver.session() as session:
    result = session.run(query).data()
    
pd.DataFrame(result)


,output
0,Number of Nodes: 46831805
1,Number of Relationships: 191363660
2,Number of Labels: 9
3,Number of Relationship Types: 10


## Summarise Node Labels

In [21]:
result = {"label": [], "count": []}

with driver.session() as session:
    labels = [row["label"] for row in session.run("CALL db.labels()")]
    for label in labels:
        query = f"MATCH (:{label}) RETURN count(*) as count"
        count = session.run(query).single()["count"]
        result["label"].append(label)
        result["count"].append(count)
        
node_labels_df = pd.DataFrame(result)
node_labels_df.sort_values(by = "count")

,label,count
7,IsilTerm,611
4,TopicTerm,212135
1,GeoName,308197
5,UniTitle,385300
6,ChronTerm,537054
2,MeetName,814044
0,CorpName,1487711
3,PerName,5087660
8,Resource,37999093


## Summarise Relationship Types


In [22]:
result = {"relType": [], "count": []}

with driver.session() as session:
    rel_types = [row["relationshipType"] for row in session.run("CALL db.relationshipTypes()")]
    for rel_type in rel_types:
        query = f"MATCH ()-[:{rel_type}]->() RETURN count(*) as count"
        count = session.run(query).single()["count"]
        result["relType"].append(rel_type)
        result["count"].append(count)
        
rel_type_df = pd.DataFrame(result)
rel_type_df.sort_values(by = "count")

,relType,count
8,RelationToUniTitle,128256
6,RelationToMeetName,422333
1,RelationToChronTerm,5446841
2,RelationToCorpName,6728127
4,RelationToGeoName,6861379
9,RelationToResource,7389423
7,RelationToPerName,20857782
3,RelationToTopicTerm,24068056
5,SocialRelation,40301595
0,RelationToIsilTerm,79159868


### Shortest Path

# Complex Queries & Data Preparation